In [22]:
import pandas as pd

# 엑셀 파일 경로
fs_path = r'C:\Users\rkddk\Downloads\financedata.xlsx'
fs_df = pd.read_excel(fs_path)

# 종목코드가 6자리가 되도록 앞에 0을 채우는 함수
def pad_stock_code(df, column_name):
    df[column_name] = df[column_name].astype(str).str.zfill(6)
    return df

# 데이터프레임에 종목코드 6자리 맞추기 적용
fs_df = pad_stock_code(fs_df, '종목코드')
fs_df.set_index('종목코드', inplace=True)
fs_df

,날짜,PER,PCR,PSR,PBR,영업이익률,ROA,매출액,영업이익,당기순이익,영업활동으로인한현금흐름
종목코드,,,,,,,,,,,
011200,2022/12,0.95,0.87,0.51,0.46,53.5,46.1,185828,99494,101171,110821
016380,2022/12,1.38,1.28,0.19,0.44,8.9,18.4,38197,3404,5348,4787
010060,2022/12,1.74,1.49,0.55,0.39,27.2,14.1,27689,7524,8778,5667
078930,2022/12,1.94,1.94,0.15,0.33,17.9,7.6,285825,51202,24827,27354
004000,2022/12,10.01,5.42,0.59,0.64,16.4,5.4,24638,4043,1462,6379
...,...,...,...,...,...,...,...,...,...,...,...
020560,2022/12,9.43,0.95,0.17,1.51,9.7,0.2,62067,5988,265,18572
001680,2022/12,9.50,3.75,0.19,0.61,3.4,2.7,40841,1400,822,-909
069960,2022/12,9.58,2.73,0.28,0.29,6.4,1.8,50141,3209,1860,4894


In [23]:
date = '2022/12'
fs_df['당기순이익점수'] = (fs_df['날짜'] == date) & (fs_df['당기순이익'] > 0)

# 결과 출력
print(fs_df)

             날짜    PER    PCR   PSR   PBR  영업이익률   ROA     매출액   영업이익   당기순이익  \
종목코드                                                                            
011200  2022/12   0.95   0.87  0.51  0.46   53.5  46.1  185828  99494  101171   
016380  2022/12   1.38   1.28  0.19  0.44    8.9  18.4   38197   3404    5348   
010060  2022/12   1.74   1.49  0.55  0.39   27.2  14.1   27689   7524    8778   
078930  2022/12   1.94   1.94  0.15  0.33   17.9   7.6  285825  51202   24827   
004000  2022/12  10.01   5.42  0.59  0.64   16.4   5.4   24638   4043    1462   
...         ...    ...    ...   ...   ...    ...   ...     ...    ...     ...   
020560  2022/12   9.43   0.95  0.17  1.51    9.7   0.2   62067   5988     265   
001680  2022/12   9.50   3.75  0.19  0.61    3.4   2.7   40841   1400     822   
069960  2022/12   9.58   2.73  0.28  0.29    6.4   1.8   50141   3209    1860   
000720  2022/12   9.59   6.64  0.18  0.51    2.7   2.3  212391   5749    4709   
004800  2022/12  90.36  11.6

In [26]:
def get_fscore(fs_df, index_date, num):
    # 날짜 필터링
    fscore_df = fs_df[fs_df['날짜'] == index_date].copy()

    # 당기순이익이 양수인지 점수 계산
    fscore_df['당기순이익점수'] = (fscore_df['당기순이익'] > 0).astype(int)

    # 영업활동으로 인한 현금흐름이 양수인지 점수 계산
    fscore_df['영업활동점수'] = (fscore_df['영업활동으로인한현금흐름'] > 0).astype(int)

    # 영업활동으로 인한 현금흐름이 당기순이익보다 큰지 점수 계산
    fscore_df['더큰영업활동점수'] = (fscore_df['영업활동으로인한현금흐름'] > fscore_df['당기순이익']).astype(int)

    # 종합점수 계산
    fscore_df['종합점수'] = fscore_df[['당기순이익점수', '영업활동점수', '더큰영업활동점수']].sum(axis=1)

    # 종합점수가 3인 항목만 필터링
    result_df = fscore_df[fscore_df['종합점수'] == 3]

    # 요청된 수량만큼 반환
    return result_df.head(num)

In [27]:
get_fscore(fs_df, '2022/12', 30)

,날짜,PER,PCR,PSR,PBR,영업이익률,ROA,매출액,영업이익,당기순이익,영업활동으로인한현금흐름,당기순이익점수,영업활동점수,더큰영업활동점수,종합점수
종목코드,,,,,,,,,,,,,,,
011200,2022/12,0.95,0.87,0.51,0.46,53.5,46.1,185828,99494,101171,110821,1,1,1,3
078930,2022/12,1.94,1.94,0.15,0.33,17.9,7.6,285825,51202,24827,27354,1,1,1,3
004000,2022/12,10.01,5.42,0.59,0.64,16.4,5.4,24638,4043,1462,6379,1,1,1,3
009150,2022/12,10.33,5.45,1.07,1.32,12.6,9.5,94246,11828,9935,15754,1,1,1,3
097950,2022/12,10.46,3.12,0.21,0.91,5.5,2.8,300795,16647,8027,16273,1,1,1,3
028260,2022/12,10.47,8.01,0.50,0.74,5.9,4.5,431617,25285,25450,26183,1,1,1,3
017670,2022/12,11.37,2.22,0.60,0.91,9.3,3.1,173050,16121,9478,51593,1,1,1,3
003090,2022/12,11.54,7.84,0.70,1.32,12.2,5.9,16973,2073,1203,1491,1,1,1,3
000120,2022/12,11.77,3.08,0.18,0.55,3.4,2.1,121307,4118,1968,6229,1,1,1,3
